# Chapter 08-04: 성능 최적화 기법

## 학습 목표
- Mixed Precision Training으로 학습 속도를 높이고 메모리를 절약한다
- `@tf.function` 데코레이터로 Python 코드를 TensorFlow 그래프로 컴파일한다
- `tf.data` 파이프라인 최적화로 데이터 병목을 제거한다
- GPU 메모리 증가 설정과 분산 학습 전략의 기초를 이해한다

## 목차
1. Mixed Precision Training
2. `@tf.function` 그래프 최적화
3. `tf.data` 파이프라인 최적화
4. GPU 메모리 설정
5. MirroredStrategy (분산 학습)
6. 정리

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import time
import os

print(f"TensorFlow 버전: {tf.__version__}")
print(f"사용 가능한 GPU: {tf.config.list_physical_devices('GPU')}")

# MNIST 데이터 로드
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test  = X_test.astype('float32') / 255.0
X_train = X_train[..., np.newaxis]
X_test  = X_test[..., np.newaxis]

## 1. Mixed Precision Training

Mixed Precision은 float16(빠른 연산)과 float32(수치 안정성)를 함께 사용한다.

- **순전파·역전파**: float16 (VRAM 절반, 텐서 코어 2배 빠름)
- **가중치 업데이트**: float32 (수치 안정성 보장)
- NVIDIA Volta(V100), Ampere(A100) 아키텍처에서 가장 큰 효과

In [ ]:
from tensorflow.keras import mixed_precision

# 현재 기본 정책 확인
print(f"기본 정책: {mixed_precision.global_policy()}")

# Mixed Precision 정책 전역 설정
# 'mixed_float16': GPU 환경
# 'mixed_bfloat16': TPU 환경 또는 일부 CPU (bfloat16은 float32와 동일한 지수 범위)
mixed_precision.set_global_policy('mixed_float16')
print(f"변경된 정책: {mixed_precision.global_policy()}")

# Mixed Precision 모델 구성 시 주의: 출력 레이어는 float32 유지
def build_mixed_precision_model():
    """Mixed Precision 학습용 모델 (출력을 float32로 강제 캐스팅)"""
    inputs = tf.keras.Input(shape=(28, 28, 1))
    x = tf.keras.layers.Conv2D(32, 3, activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D(2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    # softmax 출력: float32로 캐스팅 (수치 안정성 - 손실 함수 계산 정밀도)
    outputs = tf.keras.layers.Dense(10, activation='softmax', dtype='float32')(x)
    return tf.keras.Model(inputs, outputs)

mp_model = build_mixed_precision_model()
mp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 가중치 dtype 확인
print("\n레이어별 compute dtype:")
for layer in mp_model.layers:
    if hasattr(layer, 'compute_dtype'):
        print(f"  {layer.name:<25} compute_dtype={layer.compute_dtype}")

# 짧은 학습으로 동작 확인
mp_model.fit(X_train[:2000], y_train[:2000], epochs=2, batch_size=128, verbose=1)

# 정책을 float32로 되돌림 (이후 셀에 영향 방지)
mixed_precision.set_global_policy('float32')
print(f"\n정책 복원: {mixed_precision.global_policy()}")

## 2. `@tf.function` 그래프 최적화

`@tf.function`은 Python 함수를 TensorFlow 정적 그래프로 컴파일(트레이싱)하여 실행 속도를 높인다.

- 첫 호출 시 **트레이싱**(파이썬 코드 → 그래프 변환) 발생
- 이후 호출은 컴파일된 그래프를 재사용 → 빠름
- 입력 형상/dtype이 바뀌면 **재트레이싱** 발생 (오버헤드 주의)

In [ ]:
# 비교용 모델 생성
bench_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 일반 Python 함수 (eager 모드)
def predict_eager(images):
    """Eager 모드 추론 (그래프 컴파일 없음)"""
    return bench_model(images, training=False)

# @tf.function 데코레이터로 그래프 모드 활성화
@tf.function
def predict_graph(images):
    """그래프 모드 추론 (@tf.function 적용)"""
    return bench_model(images, training=False)

# 워밍업 (트레이싱 비용 제외)
sample = X_test[:256]
_ = predict_eager(sample)
_ = predict_graph(sample)  # 최초 호출: 트레이싱 발생

# 속도 비교 (100회 반복)
N = 100

# Eager 모드 시간 측정
start = time.perf_counter()
for _ in range(N):
    predict_eager(sample)
eager_time = (time.perf_counter() - start) / N * 1000

# Graph 모드 시간 측정
start = time.perf_counter()
for _ in range(N):
    predict_graph(sample)
graph_time = (time.perf_counter() - start) / N * 1000

print(f"Eager 모드 평균 시간: {eager_time:.3f} ms")
print(f"Graph 모드 평균 시간: {graph_time:.3f} ms")
print(f"속도 향상: {eager_time/graph_time:.2f}x")

# tf.function 사용 시 주의사항 예시
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 28, 28, 1], dtype=tf.float32)])
def predict_with_signature(images):
    """input_signature로 재트레이싱을 방지하는 패턴"""
    return bench_model(images, training=False)

result = predict_with_signature(sample)
print(f"\ninput_signature 버전 결과 형상: {result.shape}")

## 3. `tf.data` 파이프라인 최적화

데이터 로딩이 학습보다 느리면 GPU가 유휴 상태가 된다.
`tf.data`의 최적화 기법으로 데이터 병목을 제거한다.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE  # TensorFlow가 자동으로 병렬화 수준 결정

# 데이터 증강 레이어 (tf.data map에서 사용)
augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# ── 비최적화 파이프라인 (비교 기준) ──────────────────────────────
def make_baseline_dataset(images, labels, batch_size=128):
    """최적화 없는 기본 파이프라인 (순차 처리)"""
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    ds = ds.shuffle(1000)
    ds = ds.map(lambda x, y: (augmentation(x[tf.newaxis], training=True)[0], y))
    ds = ds.batch(batch_size)
    return ds

# ── 최적화 파이프라인 ───────────────────────────────────────────
def make_optimized_dataset(images, labels, batch_size=128):
    """
    최적화 파이프라인:
    - cache()       : 첫 에포크 후 데이터를 메모리에 캐시 (디스크 I/O 제거)
    - map + AUTOTUNE: 병렬 전처리 (CPU 코어 자동 활용)
    - prefetch()    : GPU 연산 중 다음 배치 미리 준비 (CPU-GPU 파이프라이닝)
    """
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    ds = ds.cache()  # 데이터를 메모리에 캐시
    ds = ds.shuffle(1000)
    # num_parallel_calls=AUTOTUNE: 병렬 처리 수준 자동 결정
    ds = ds.map(
        lambda x, y: (augmentation(x[tf.newaxis], training=True)[0], y),
        num_parallel_calls=AUTOTUNE
    )
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTOTUNE)  # 다음 배치 비동기 준비
    return ds

# 파이프라인 구성
baseline_ds  = make_baseline_dataset(X_train[:5000], y_train[:5000])
optimized_ds = make_optimized_dataset(X_train[:5000], y_train[:5000])

# 파이프라인 요소 수 확인
print(f"배치 수 (baseline) : {sum(1 for _ in baseline_ds)}")
print(f"배치 수 (optimized): {sum(1 for _ in optimized_ds)}")

# 간단한 모델로 속도 비교
def make_train_model():
    m = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    m.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    return m

# 비최적화 파이프라인 학습 시간
m1 = make_train_model()
start = time.perf_counter()
m1.fit(baseline_ds, epochs=2, verbose=0)
baseline_time = time.perf_counter() - start

# 최적화 파이프라인 학습 시간
m2 = make_train_model()
start = time.perf_counter()
m2.fit(optimized_ds, epochs=2, verbose=0)
optimized_time = time.perf_counter() - start

print(f"\n비최적화 파이프라인: {baseline_time:.2f}초")
print(f"최적화 파이프라인 : {optimized_time:.2f}초")
print(f"속도 향상         : {baseline_time/optimized_time:.2f}x")

## 4. GPU 메모리 증가 설정

기본적으로 TensorFlow는 사용 가능한 GPU 메모리 전체를 할당한다.
`set_memory_growth(True)`를 설정하면 실제 필요한 만큼만 점진적으로 할당한다.

In [ ]:
# GPU 디바이스 목록 조회
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # 각 GPU에 대해 메모리 증가 설정 활성화
        # 주의: 반드시 다른 TF 연산 이전에 설정해야 한다
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"{len(gpus)}개 GPU에 memory growth 설정 완료")

        # 특정 GPU에 최대 메모리 한도 설정 (선택 사항)
        # tf.config.set_logical_device_configuration(
        #     gpus[0],
        #     [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]  # 4GB
        # )
    except RuntimeError as e:
        # 이미 TF 연산이 실행된 후에는 설정 변경 불가
        print(f"메모리 설정 오류 (이미 초기화됨): {e}")
else:
    print("GPU를 찾을 수 없습니다. CPU 환경에서 실행 중입니다.")
    print("권장 스크립트 시작부에 아래 코드를 배치하세요:")
    print("""
import tensorflow as tf

# GPU 메모리 증가 설정 (프로그램 시작 시 최초로 실행)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    """)

# CPU/GPU 디바이스 정보 출력
print("\n현재 가용 디바이스:")
for d in tf.config.list_logical_devices():
    print(f"  {d.device_type}: {d.name}")

## 5. MirroredStrategy - 멀티 GPU 분산 학습

`tf.distribute.MirroredStrategy`는 단일 머신의 여러 GPU에 학습을 분산한다.

- 각 GPU에 모델의 **복사본(replica)**을 배치
- 배치를 GPU 수만큼 분할하여 병렬 처리
- 그래디언트는 **All-Reduce** 알고리즘으로 모든 GPU에서 동기화
- `strategy.scope()` 내부에서 모델 생성 및 컴파일만 하면 나머지는 자동

In [ ]:
# MirroredStrategy 초기화
# GPU가 없으면 자동으로 CPU로 폴백
strategy = tf.distribute.MirroredStrategy()
print(f"복제본(Replica) 수: {strategy.num_replicas_in_sync}")
print(f"(GPU가 없으면 1, GPU 2개면 2로 표시)")

# 분산 학습의 핵심: strategy.scope() 내에서 모델 생성
with strategy.scope():
    # 이 블록 안에서 생성된 변수들은 모든 GPU에 미러링됨
    dist_model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    dist_model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

# 분산 학습 시 배치 크기는 전체 배치 크기
# (GPU당 배치 크기 × GPU 수 = 전체 배치 크기)
GLOBAL_BATCH_SIZE = 128 * strategy.num_replicas_in_sync
print(f"\n전체 배치 크기: {GLOBAL_BATCH_SIZE}")

# 실제 학습 (MirroredStrategy는 model.fit과 완전 호환)
dist_model.fit(
    X_train[:5000], y_train[:5000],
    validation_data=(X_test[:1000], y_test[:1000]),
    batch_size=GLOBAL_BATCH_SIZE,
    epochs=2,
    verbose=1
)

print("\n=== 다른 분산 전략 소개 ===")
print("MirroredStrategy     : 단일 머신 멀티 GPU (동기식)")
print("MultiWorkerMirrored  : 멀티 머신 멀티 GPU (동기식)")
print("TPUStrategy          : Google TPU 분산 학습")
print("ParameterServerStrategy: 파라미터 서버 방식 (비동기식)")

## 6. 정리

### 성능 최적화 기법 요약

| 기법 | 효과 | 적용 난이도 | 주요 API |
|------|------|-------------|----------|
| Mixed Precision | VRAM 절반, 속도 1.5–3x | 낮음 | `set_global_policy('mixed_float16')` |
| `@tf.function` | 추론 속도 향상 | 낮음 | `@tf.function` 데코레이터 |
| `tf.data` 최적화 | 데이터 병목 제거 | 중간 | `cache()`, `prefetch()`, `map(num_parallel_calls=AUTOTUNE)` |
| 메모리 증가 | OOM 방지 | 낮음 | `set_memory_growth(True)` |
| MirroredStrategy | 선형 스케일 학습 | 낮음 | `strategy.scope()` |

### 최적화 체크리스트

```
[ ] GPU 메모리 증가 설정 (프로그램 시작 시)
[ ] tf.data 파이프라인에 cache() + prefetch(AUTOTUNE) 적용
[ ] map()에 num_parallel_calls=AUTOTUNE 설정
[ ] Mixed Precision 활성화 (GPU가 있는 경우)
[ ] 커스텀 학습 스텝에 @tf.function 적용
[ ] 멀티 GPU 환경에서 MirroredStrategy 사용
```